# Inventory Consolidation Analysis

This notebook performs inventory rebalancing analysis using sales and stock data to generate movement recommendations between stores and warehouses.

In [17]:
import pandas as pd
import numpy as np
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


In [1]:
import pandas as pd
import numpy as np

# Load the datasets
print("Loading datasets...")

df_sales = pd.read_csv(r"C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\sales.csv")
df_soh = pd.read_csv(r"C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\Stock Report Batchwise (59).csv")
df_format = pd.read_csv(r"C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\Consolidation format.csv")

# Standardize column names: strip whitespace
df_sales.columns = df_sales.columns.str.strip()
df_soh.columns = df_soh.columns.str.strip()
df_format.columns = df_format.columns.str.strip()

print(f"Sales data loaded: {df_sales.shape}")
print(f"SOH data loaded: {df_soh.shape}")
print(f"Format template loaded: {df_format.shape}")

print("\nSales data columns:", df_sales.columns.tolist())
print("SOH data columns:", df_soh.columns.tolist())
print("Format template columns:", df_format.columns.tolist())

Loading datasets...


C:\Users\Shivansh Pal\AppData\Local\Temp\ipykernel_11064\907951284.py:7: DtypeWarning: Columns (51,63,83,143,158,171,172,180,181,203,233) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sales = pd.read_csv(r"C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\sales.csv")


Sales data loaded: (169581, 237)
SOH data loaded: (74899, 111)
Format template loaded: (5, 8)

Sales data columns: ['RegionName', 'StoreName', 'AlternateStoreCode', 'StoreGSTNumber', 'BillDate', 'BillTime', 'BillNumber', 'CustomerCode', 'CustomerName', 'CustomerGSTNumber', 'CustomerMobile', 'Quantity', 'FreeQty', 'FreeMRPValue', 'TaxDescription', 'BaseValue', 'OtherTax', 'TNGSTAmount', 'CST', 'SGST', 'CGST', 'IGST', 'UTGST', 'Tax', 'IsTaxInclusive', 'TaxTransactionNumber', 'HSNCode', 'Amount', 'ProductName', 'LanguageProductName', 'BrandName', 'SupplierRefCode', 'CategoryCode', 'Department', 'SubCategory', 'ProductAttribute2', 'ProductAttribute3', 'ProductAttribute4', 'ProductAttribute5', 'ProductCode', 'EANCode', 'AlternateProductCodes', 'Batch', 'MRP', 'ProductLevelDisc%', 'ProductLevelDiscAmount', 'BillLevelDiscAmount', 'BillLevelDisc%', 'TotalDiscAmount', 'TotalDisc%', 'OtherDiscountPercentage', 'ExpiryDate', 'SalesmanCode', 'SalesmanName', 'Classification', 'Cenvat', 'Educess', 'S

C:\Users\Shivansh Pal\AppData\Local\Temp\ipykernel_11064\907951284.py:8: DtypeWarning: Columns (25,47,91,97,99,104) have mixed types. Specify dtype option on import or set low_memory=False.
  df_soh = pd.read_csv(r"C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\Stock Report Batchwise (59).csv")


In [2]:
# Check for required columns and raise error if missing

required_sales_cols = ['StoreName', 'ProductCode', 'ProductName', 'Quantity']
required_soh_cols = ['StoreName', 'ProductCode', 'ProductName', 'ActualStock']
required_format_cols = ['Part No']  # Only require 'Part No' in the format file

for col in required_sales_cols:
    if col not in df_sales.columns:
        raise ValueError(f"Missing column in sales.csv: {col}")
for col in required_soh_cols:
    if col not in df_soh.columns:
        raise ValueError(f"Missing column in Stock Report: {col}")
for col in required_format_cols:
    if col not in df_format.columns:
        raise ValueError(f"Missing column in format file: {col}")

print("All required columns found in all files.")

All required columns found in all files.


In [3]:
# Check for warehouses in both datasets
print("Checking for warehouses in datasets...")

sales_stores = df_sales['StoreName'].unique()
soh_stores = df_soh['StoreName'].unique()

sales_warehouses = [store for store in sales_stores if 'warehouse' in str(store).lower()]
soh_warehouses = [store for store in soh_stores if 'warehouse' in str(store).lower()]

print(f"Warehouses in sales data: {len(sales_warehouses)}")
print(f"Warehouses in SOH data: {len(soh_warehouses)}")

if soh_warehouses:
    print("\nSample warehouses from SOH data:")
    for i, wh in enumerate(soh_warehouses[:5]):
        print(f"  {i+1}. {wh}")

print(f"\nTotal unique stores in sales: {len(sales_stores)}")
print(f"Total unique stores in SOH: {len(soh_stores)}")

Checking for warehouses in datasets...
Warehouses in sales data: 0
Warehouses in SOH data: 7

Sample warehouses from SOH data:
  1. Warehouse Bose2- R T Nagar Bengaluru
  2. Warehouse Bose 2- Lakshmipuram Thiruvanmiyur Chennai
  3. Warehouse Bose 2- Punjagutta Hyderabad
  4. Warehouse Imagine 3- Jayanagar Bengaluru
  5. Warehouse Imagine 3- Lakshmipuram Thiruvanmiyur Chennai

Total unique stores in sales: 100
Total unique stores in SOH: 110


In [4]:
# Data cleaning and preparation
print("Cleaning and preparing data...")

# Clean sales data - remove Apple products
def clean_data(df):
    brand_col = 'BrandName' if 'BrandName' in df.columns else 'Brand'
    if brand_col in df.columns:
        initial_count = len(df)
        df = df[df[brand_col] != 'Apple']
        print(f"  Removed {initial_count - len(df)} Apple products")
    return df

df_sales_clean = clean_data(df_sales.copy())

# Prepare sales data - aggregate by store and product
sales_agg = df_sales_clean.groupby(['StoreName', 'ProductDescription'], as_index=False).agg({
    'Quantity': 'sum',
    'Amount': 'sum'
}).rename(columns={
    'StoreName': 'Store',
    'ProductDescription': 'Product', 
    'Quantity': 'Total_Quantity_Sold',
    'Amount': 'Sales'
})

# Prepare SOH data - aggregate by store and product
soh_agg = df_soh.groupby(['StoreName', 'ProductDescription'], as_index=False).agg({
    'ActualStock': 'sum',
    'ProductName': 'first'  # Keep product name for mapping
}).rename(columns={
    'StoreName': 'Store',
    'ProductDescription': 'Product',
    'ActualStock': 'Stock'
})

print(f"Sales aggregated: {sales_agg.shape[0]} store-product combinations")
print(f"SOH aggregated: {soh_agg.shape[0]} store-product combinations")

# Convert numeric columns
sales_agg['Total_Quantity_Sold'] = pd.to_numeric(sales_agg['Total_Quantity_Sold'], errors='coerce').fillna(0)
sales_agg['Sales'] = pd.to_numeric(sales_agg['Sales'], errors='coerce').fillna(0)
soh_agg['Stock'] = pd.to_numeric(soh_agg['Stock'], errors='coerce').fillna(0)

print("Data types converted to numeric")

Cleaning and preparing data...
  Removed 88243 Apple products
Sales aggregated: 14980 store-product combinations
SOH aggregated: 41893 store-product combinations
Data types converted to numeric


In [5]:
# Merge sales and stock data
print("Merging sales and stock data...")

# Merge on store and product
df_merged = pd.merge(sales_agg, soh_agg, on=['Store', 'Product'], how='outer')
df_merged['Total_Quantity_Sold'] = df_merged['Total_Quantity_Sold'].fillna(0)
df_merged['Sales'] = df_merged['Sales'].fillna(0)
df_merged['Stock'] = df_merged['Stock'].fillna(0)

print(f"Merged data shape: {df_merged.shape}")
print(f"Total products: {df_merged['Product'].nunique()}")
print(f"Total stores: {df_merged['Store'].nunique()}")

# Display sample merged data
print("\nSample merged data:")
print(df_merged.head())

Merging sales and stock data...
Merged data shape: (48141, 6)
Total products: 4955
Total stores: 112

Sample merged data:
                          Store      Product  Total_Quantity_Sold     Sales  \
0  Bose- Ambience Mall Gurugram        24643                  5.0  235907.5   
1  Bose- Ambience Mall Gurugram        24644                  0.0       0.0   
2  Bose- Ambience Mall Gurugram  722139-0010                  3.0   25200.0   
3  Bose- Ambience Mall Gurugram  722139-0020                  0.0       0.0   
4  Bose- Ambience Mall Gurugram  722140-0020                  0.0       0.0   

   Stock                          ProductName  
0    1.0  251 ENVIRONMENTAL BLK WITH BLK BRKT  
1    1.0  251 ENVIRONMENTAL WHT WITH WHT BRKT  
2    0.0                                  NaN  
3    1.0   UFS 20 II UNIVERSAL FLOORSTAND WHT  
4    1.0  UTS-20 II UNIVERSAL TABLE STAND WHT  


In [6]:
# Demand forecasting
print("Calculating demand forecasts...")

# Calculate monthly sale rate (assuming 3-month data)
n_months = 3
df_merged['Monthly_Sale_Rate'] = df_merged['Total_Quantity_Sold'] / n_months
df_merged['Forecast_Demand'] = df_merged['Monthly_Sale_Rate'] * n_months

# Cap unrealistic forecasts
df_merged['Monthly_Sale_Rate'] = df_merged['Monthly_Sale_Rate'].clip(upper=100)
df_merged['Forecast_Demand'] = df_merged['Monthly_Sale_Rate'] * n_months

print(f"Average monthly sale rate: {df_merged['Monthly_Sale_Rate'].mean():.2f}")
print(f"Average forecast demand: {df_merged['Forecast_Demand'].mean():.2f}")
print(f"Total current stock: {df_merged['Stock'].sum():.0f}")
print(f"Total forecasted demand: {df_merged['Forecast_Demand'].sum():.0f}")

Calculating demand forecasts...
Average monthly sale rate: 0.49
Average forecast demand: 1.47
Total current stock: 219263
Total forecasted demand: 70610


In [8]:
from collections import defaultdict
# Classify stores and warehouses
print("Classifying stores and warehouses...")

# Create lookup dictionaries
stock_lookup = dict(zip(zip(df_merged['Store'], df_merged['Product']), df_merged['Stock']))
rate_lookup = dict(zip(zip(df_merged['Store'], df_merged['Product']), df_merged['Monthly_Sale_Rate']))
sales_lookup = dict(zip(zip(df_merged['Store'], df_merged['Product']), df_merged['Total_Quantity_Sold']))

# Product name mapping
product_name_map = df_merged.dropna(subset=['ProductName']).drop_duplicates(subset=['Product'])[['Product', 'ProductName']].set_index('Product')['ProductName'].to_dict()
print(f"Product name mappings: {len(product_name_map)}")

# Classify warehouses vs retail stores
warehouse_by_prod = defaultdict(list)
stores_by_prod = defaultdict(list)

for store, prod in stock_lookup:
    store_lower = str(store).lower()
    is_warehouse = (
        'warehouse' in store_lower or 
        'wh-' in store_lower or 
        'depot' in store_lower or
        'distribution' in store_lower or
        'central' in store_lower or
        store_lower.startswith('wh ')
    )
    
    if is_warehouse:
        warehouse_by_prod[prod].append(store)
    else:
        stores_by_prod[prod].append(store)

warehouse_count = len(set([store for stores in warehouse_by_prod.values() for store in stores]))
retail_count = len(set([store for stores in stores_by_prod.values() for store in stores]))

print(f"Warehouses identified: {warehouse_count}")
print(f"Retail stores identified: {retail_count}")

if warehouse_count > 0:
    sample_warehouses = list(set([store for stores in warehouse_by_prod.values() for store in stores]))[:5]
    print("Sample warehouses:")
    for wh in sample_warehouses:
        print(f"  - {wh}")
else:
    print("\nWarning: No warehouses identified by naming pattern.")
    print("Using stores with highest stock as warehouses...")
    
    # Use top stores by average stock as warehouses
    store_avg_stock = df_merged.groupby('Store')['Stock'].mean().sort_values(ascending=False)
    warehouse_threshold = max(5, len(store_avg_stock) // 20)  # Top 5% or at least 5 stores
    warehouse_stores = store_avg_stock.head(warehouse_threshold).index.tolist()
    
    print(f"Using top {len(warehouse_stores)} stores by stock as warehouses")
    
    # Reassign based on stock levels
    warehouse_by_prod = defaultdict(list)
    stores_by_prod = defaultdict(list)
    
    for store, prod in stock_lookup:
        if store in warehouse_stores:
            warehouse_by_prod[prod].append(store)
        else:
            stores_by_prod[prod].append(store)
    
    warehouse_count = len(set([store for stores in warehouse_by_prod.values() for store in stores]))
    retail_count = len(set([store for stores in stores_by_prod.values() for store in stores]))
    
    print(f"Reassigned - Warehouses: {warehouse_count}, Retail stores: {retail_count}")

Classifying stores and warehouses...
Product name mappings: 4528
Warehouses identified: 7
Retail stores identified: 105
Sample warehouses:
  - Warehouse Bose2- R T Nagar Bengaluru
  - Warehouse Bose 2- Punjagutta Hyderabad
  - Warehouse Imagine 3- Jayanagar Bengaluru
  - Warehouse Bose 2- Lakshmipuram Thiruvanmiyur Chennai
  - Warehouse- Tech 4 Jayanagar Bengaluru


In [9]:
# Calculate product movements
print("Calculating product movements...")

movements = []
processed_combinations = set()

# Copy stock lookup for modification during movement calculation
available_stock = stock_lookup.copy()

for _, row in df_merged.iterrows():
    dest = row['Store']
    prod = row['Product']
    
    # Skip if already processed
    if (dest, prod) in processed_combinations:
        continue
    processed_combinations.add((dest, prod))
    
    current_stock = float(available_stock.get((dest, prod), 0))
    forecast_demand = float(row['Forecast_Demand'])
    demand_gap = forecast_demand - current_stock
    
    if demand_gap <= 0:
        continue
    
    remaining_demand = demand_gap
    
    # First try to fulfill from warehouses
    for src in warehouse_by_prod.get(prod, []):
        if remaining_demand <= 0 or src == dest:
            continue
            
        available = float(available_stock.get((src, prod), 0))
        transfer_qty = min(available, remaining_demand)
        
        if transfer_qty > 0:
            movements.append({
                'ProductName': product_name_map.get(prod, prod),
                'Product': prod,
                'Source': src,
                'Destination': dest,
                'Quantity': transfer_qty
            })
            available_stock[(src, prod)] = max(0, available_stock.get((src, prod), 0) - transfer_qty)
            remaining_demand -= transfer_qty
    
    # Then try from other retail stores (prioritize low-velocity stores)
    if remaining_demand > 0:
        available_stores = [s for s in stores_by_prod.get(prod, []) if s != dest]
        # Sort by monthly sale rate (ascending) to prioritize low-velocity stores
        sorted_stores = sorted(available_stores, key=lambda s: rate_lookup.get((s, prod), 0))
        
        for src in sorted_stores:
            if remaining_demand <= 0:
                break
                
            available = float(available_stock.get((src, prod), 0))
            transfer_qty = min(available, remaining_demand)
            
            if transfer_qty > 0:
                movements.append({
                    'ProductName': product_name_map.get(prod, prod),
                    'Product': prod,
                    'Source': src,
                    'Destination': dest,
                    'Quantity': transfer_qty
                })
                available_stock[(src, prod)] = max(0, available_stock.get((src, prod), 0) - transfer_qty)
                remaining_demand -= transfer_qty

print(f"Total movements calculated: {len(movements)}")
if movements:
    total_qty = sum(m['Quantity'] for m in movements)
    print(f"Total quantity to be moved: {total_qty:.0f}")

Calculating product movements...
Total movements calculated: 7979
Total quantity to be moved: 23294


In [10]:
# Create movement DataFrame
if movements:
    mov_df = pd.DataFrame(movements)
    
    print(f"Movement DataFrame created: {mov_df.shape}")
    print(f"Products involved: {mov_df['Product'].nunique()}")
    print(f"Source locations: {mov_df['Source'].nunique()}")
    print(f"Destination locations: {mov_df['Destination'].nunique()}")
    
    # Check warehouse vs retail sources
    warehouse_movements = mov_df[mov_df['Source'].str.lower().str.contains('warehouse', na=False)]
    retail_movements = mov_df[~mov_df['Source'].str.lower().str.contains('warehouse', na=False)]
    
    print(f"\nMovements from warehouses: {len(warehouse_movements)} ({warehouse_movements['Quantity'].sum():.0f} units)")
    print(f"Movements from retail stores: {len(retail_movements)} ({retail_movements['Quantity'].sum():.0f} units)")
    
    print("\nTop 10 largest movements:")
    top_movements = mov_df.nlargest(10, 'Quantity')[['ProductName', 'Source', 'Destination', 'Quantity']]
    for _, row in top_movements.iterrows():
        print(f"  {row['ProductName'][:50]}... {row['Quantity']:.0f} units: {row['Source']} → {row['Destination']}")
else:
    print("No movements needed - inventory appears well balanced")
    mov_df = pd.DataFrame(columns=['ProductName', 'Product', 'Source', 'Destination', 'Quantity'])

Movement DataFrame created: (7979, 5)
Products involved: 1151
Source locations: 107
Destination locations: 97

Movements from warehouses: 1933 (7301 units)
Movements from retail stores: 6046 (15993 units)

Top 10 largest movements:
  TEKNE Spotfree Cleaning Spray... 225 units: Warehouse Imagine 3- Jayanagar Bengaluru → Imagine- Lulu Mall Kochi
  TEKNE Spotfree Cleaning Spray... 187 units: Warehouse Imagine 3- Jayanagar Bengaluru → Imagine- Nexus Koramangala Bengaluru
  TEKNE Spotfree Cleaning Spray... 121 units: Warehouse Imagine 3- Jayanagar Bengaluru → Imagine- HSR Layout Bengaluru
  Jute Bag Large - APR... 100 units: Bose- GIP Noida → Imagine- Express Avenue Chennai
  Jute Bag Large - APR... 100 units: Bose- Inorbit Mall Malad Mumbai → Imagine- HSR Layout Bengaluru
  Jute Bag Large - APR... 100 units: Bose- Inorbit Mall Vashi Mumbai → Imagine- HSR Layout Bengaluru
  Jute Bag Large - APR... 100 units: Bose- Khan Marlet New Delhi → Imagine- Koramangala Bengaluru
  Jute Bag Large - APR

In [15]:
# Format output according to consolidation format
print("Creating consolidation format output...")

# Required columns: Part No, ProductName, From Store, To Store, From Store Current SOH, To store Current SOH, From STORE sales, TO store Sales, Sugested Qty transfer

consolidation_results = []

# Use the same source as your movement calculation (movements or mov_df)
# If you have a DataFrame: use mov_df.iterrows()
# If you have a list of dicts: use for m in movements
movement_source = mov_df if 'mov_df' in locals() else movements

for row in (movement_source.iterrows() if isinstance(movement_source, pd.DataFrame) else enumerate(movement_source)):
    if isinstance(movement_source, pd.DataFrame):
        row = row[1]  # get the Series from (idx, Series)
    # If using dicts, row is already the dict

    part_no = row['Product'] if 'Product' in row else row['Part No']
    product_name = row.get('ProductName', part_no)
    from_store = row['Source'] if 'Source' in row else row['From Store']
    to_store = row['Destination'] if 'Destination' in row else row['To Store']
    suggested_qty = row['Quantity'] if 'Quantity' in row else row['Sugested Qty transfer']

    # Get current SOH for both stores
    from_soh = stock_lookup.get((from_store, part_no), 0)
    to_soh = stock_lookup.get((to_store, part_no), 0)

    # Get sales for both stores
    from_sales = sales_lookup.get((from_store, part_no), 0)
    to_sales = sales_lookup.get((to_store, part_no), 0)

    consolidation_results.append({
        'Part No': part_no,
        'ProductName': product_name,
        'From Store': from_store,
        'To Store': to_store,
        'From Store Current SOH': from_soh,
        'To store Current SOH': to_soh,
        'From STORE sales': from_sales,
        'TO store Sales': to_sales,
        'Sugested Qty transfer': suggested_qty
    })

df_format_result = pd.DataFrame(consolidation_results)

print(f"Consolidation format created: {df_format_result.shape}")

if not df_format_result.empty:
    print(f"Total movements: {len(df_format_result)}")
    print(f"Total quantity to transfer: {df_format_result['Sugested Qty transfer'].sum():.0f}")

    print("\nSample consolidation format output:")
    print(df_format_result.head())
else:
    print("No consolidation movements generated")

Creating consolidation format output...
Consolidation format created: (7979, 9)
Total movements: 7979
Total quantity to transfer: 23294

Sample consolidation format output:
       Part No                          ProductName  \
0        24643  251 ENVIRONMENTAL BLK WITH BLK BRKT   
1        24643  251 ENVIRONMENTAL BLK WITH BLK BRKT   
2        24643  251 ENVIRONMENTAL BLK WITH BLK BRKT   
3        24643  251 ENVIRONMENTAL BLK WITH BLK BRKT   
4  722139-0010   UFS-20 II UNIVERSAL FLOORSTAND BLK   

                                     From Store                      To Store  \
0                Bose- Bharatiya City Bengaluru  Bose- Ambience Mall Gurugram   
1              Bose- Emporium MG Road Bengaluru  Bose- Ambience Mall Gurugram   
2                  Bose- Inorbit Mall Hyderabad  Bose- Ambience Mall Gurugram   
3               Bose- Inorbit Mall Malad Mumbai  Bose- Ambience Mall Gurugram   
4  Bose- Vintage Boulevard Somajiguda Hyderabad  Bose- Ambience Mall Gurugram   

   From S

In [12]:
# Save results
print("Saving results...")

# Save consolidation format
output_path_consolidation = r'C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\consolidation_result.csv'
df_format_result.to_csv(output_path_consolidation, index=False)

# Save detailed movements
output_path_movements = r'C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\detailed_movements.csv'
mov_df.to_csv(output_path_movements, index=False)

# Save analysis summary
summary_data = {
    'Total_Products_Processed': [df_merged['Product'].nunique()],
    'Total_Stores_Analyzed': [df_merged['Store'].nunique()],
    'Warehouses_Identified': [warehouse_count],
    'Retail_Stores_Identified': [retail_count],
    'Total_Movements_Recommended': [len(df_format_result)],
    'Total_Quantity_To_Transfer': [df_format_result['Sugested Qty transfer'].sum() if not df_format_result.empty else 0],
    'Total_Current_Stock': [df_merged['Stock'].sum()],
    'Total_Forecasted_Demand': [df_merged['Forecast_Demand'].sum()]
}

summary_df = pd.DataFrame(summary_data)
summary_path = r'C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\analysis_summary.csv'
summary_df.to_csv(summary_path, index=False)

print("Files saved successfully:")
print(f"  - Consolidation format: {output_path_consolidation}")
print(f"  - Detailed movements: {output_path_movements}")
print(f"  - Analysis summary: {summary_path}")

print("\n=== CONSOLIDATION ANALYSIS COMPLETE ===")
if not df_format_result.empty:
    print(f"✓ {len(df_format_result)} movements recommended")
    print(f"✓ {df_format_result['Sugested Qty transfer'].sum():.0f} total units to transfer")
    print(f"✓ {df_format_result['Part No'].nunique()} products involved")
    print(f"✓ {df_format_result['To Store'].nunique()} destination stores")
else:
    print("✓ No movements needed - inventory is well balanced")

Saving results...
Files saved successfully:
  - Consolidation format: C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\consolidation_result.csv
  - Detailed movements: C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\detailed_movements.csv
  - Analysis summary: C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\analysis_summary.csv

=== CONSOLIDATION ANALYSIS COMPLETE ===
✓ 7979 movements recommended
✓ 23294 total units to transfer
✓ 1151 products involved
✓ 97 destination stores


In [13]:
# Create Excel report from the perfect CSV file
print("Creating Excel report from consolidation_result.csv...")

# Read the perfect CSV file
csv_path = r'C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\consolidation_result.csv'
excel_path = r'C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\consolidation_report.xlsx'

# Read the CSV
df_excel = pd.read_csv(csv_path)

# Create Excel writer with formatting
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    # Write main consolidation data
    df_excel.to_excel(writer, sheet_name='Consolidation_Report', index=False)
    
    # Get workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets['Consolidation_Report']
    
    # Add formatting
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    
    number_format = workbook.add_format({'num_format': '#,##0'})
    
    # Format headers
    for col_num, value in enumerate(df_excel.columns.values):
        worksheet.write(0, col_num, value, header_format)
        
    # Auto-adjust column widths
    for i, col in enumerate(df_excel.columns):
        column_len = max(df_excel[col].astype(str).str.len().max(), len(col) + 2)
        worksheet.set_column(i, i, min(column_len, 50))
    
    # Format number columns
    qty_col = df_excel.columns.get_loc('Sugested Qty transfer')
    soh_cols = [df_excel.columns.get_loc('From Store Current SOH'), 
                df_excel.columns.get_loc('To store Current SOH')]
    sales_cols = [df_excel.columns.get_loc('From STORE sales'), 
                  df_excel.columns.get_loc('TO store Sales')]
    
    for col in [qty_col] + soh_cols + sales_cols:
        worksheet.set_column(col, col, 15, number_format)
    
    # Add summary sheet
    summary_data = {
        'Metric': [
            'Total Movements',
            'Total Quantity to Transfer',
            'Products Involved',
            'Source Locations',
            'Destination Locations',
            'Average Transfer per Movement'
        ],
        'Value': [
            len(df_excel),
            df_excel['Sugested Qty transfer'].sum(),
            df_excel['Part No'].nunique(),
            df_excel['From Store'].nunique(),
            df_excel['To Store'].nunique(),
            df_excel['Sugested Qty transfer'].mean()
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    # Format summary sheet
    summary_worksheet = writer.sheets['Summary']
    summary_worksheet.set_column(0, 0, 30)
    summary_worksheet.set_column(1, 1, 20)

print(f"Excel report created successfully: {excel_path}")
print(f"File contains {len(df_excel)} movements across {len(df_excel.columns)} columns")
print("Sheets created: 'Consolidation_Report' and 'Summary'")

Creating Excel report from consolidation_result.csv...
Excel report created successfully: C:\Users\Shivansh Pal\Desktop\Planogramm\consolidation dataset\consolidation_report.xlsx
File contains 7979 movements across 9 columns
Sheets created: 'Consolidation_Report' and 'Summary'
